# **GPT Action ライブラリ: Google Drive**

## **はじめに**

このページは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解してください：

* [GPT Actionsの紹介](https://platform.openai.com/docs/actions/introduction)
* [GPT Actions ライブラリの紹介](https://platform.openai.com/docs/actions/actions-library)
* [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、Googleのファイルストレージシステムである**Google Drive**への接続方法の概要を提供します。このアクションにより、ファイル名の一覧表示と検索、ファイルコンテンツのGPTへの読み込み、そして最終的にそのデータをChatGPTのコンテキストとして使用することができます。このアクションセットは、[Google Drive API](https://developers.google.com/drive/api/guides/about-sdk)で見つかる追加メソッドによって拡張可能です。以下のような小さなファイルを読み取ることができる汎用GPTが必要な場合に最適です：

* 会議議事録
* 製品設計文書
* 短いメモ
* よくある質問

書籍全体や多くの行を持つ複雑なCSVなど、より長いメモを読み取りたい場合は、Google DocsまたはGoogle Sheets専用のGPTを構築することをお勧めします。

### 価値 + ビジネスケースの例

ユーザーは、ChatGPTの自然言語機能を活用してGoogle Drive内のファイルに直接接続できるようになります。

使用例：

- ユーザーが特定のトピックに関連するファイルを検索する必要がある場合
- ユーザーが文書の奥深くに埋もれた重要な質問への回答が必要な場合

## **アプリケーション情報**

### **アプリケーションの主要リンク**

開始する前に、アプリケーションの以下のリンクを確認してください：

* アプリケーションWebサイト: [https://www.google.com/drive/](https://www.google.com/drive/)
* アプリケーションAPI文書: [https://developers.google.com/drive/api/guides/about-sdk](https://developers.google.com/drive/api/guides/about-sdk)

### **アプリケーションの前提条件**

開始する前に、Google Cloudアカウントを持ち、Drive APIが有効になっていることを確認してください：

* Google Cloudプロジェクトをセットアップする
* Google APIライブラリからGoogle Drive APIを有効にする
* アプリケーションの「公開ステータス」が「テスト中」の場合、ユーザーがアプリケーションに追加されていることを確認する

## **ChatGPTの手順**

### **カスタムGPT指示の例**

カスタムGPTを作成したら、開始するために以下のテキストを指示パネルにコピーしてください。あなたの使用ケースに特有の追加コンテキストを追加する必要があるかもしれません。この方法で、明確性と正確性を最適化するために追加する指示をテストすることが価値があります。質問がありますか？この手順がより詳細にどのように機能するかを確認するには、[開始例](https://platform.openai.com/docs/actions/getting-started)をチェックしてください。

In [ ]:
*** Context *** 

You are an office helper who takes a look at files within Google Drive and reads in information.  In this way, when asked about something, please take a look at all of the relevant information within the drive.  Respect file names, but also take a look at each document and sheet.

*** Instructions ***

Use the 'listFiles' function to get a list of files available within docs.  In this way, determine out of this list which files make the most sense for you to pull back taking into account name and title.  After the output of listFiles is called into context, act like a normal business analyst.  Things you could be asked to be are:

- Summaries: what happens in a given file?  Please give a consistent, concise answer and read through the entire file before giving an answer.
- Professionalism: Behave professionally, providing clear and concise responses.
- Synthesis, Coding, and Data Analysis: ensure coding blocks are explained.
- When handling dates: make sure that dates are searched using date fields and also if you don't find anything, use titles.
- Clarification: Ask for clarification when needed to ensure accuracy and completeness in fulfilling user requests.  Try to make sure you know exactly what is being asked. 
- Privacy and Security: Respect user privacy and handle all data securely.

*** Examples of Documentation ***
Here is the relevant query documentation from Google for the listFiles function:
What you want to query	Example
Files with the name "hello"	name = 'hello'
Files with a name containing the words "hello" and "goodbye"	name contains 'hello' and name contains 'goodbye'
Files with a name that does not contain the word "hello"	not name contains 'hello'
Folders that are Google apps or have the folder MIME type	mimeType = 'application/vnd.google-apps.folder'
Files that are not folders	mimeType != 'application/vnd.google-apps.folder'
Files that contain the text "important" and in the trash	fullText contains 'important' and trashed = true
Files that contain the word "hello"	fullText contains 'hello'
Files that do not have the word "hello"	not fullText contains 'hello'
Files that contain the exact phrase "hello world"	fullText contains '"hello world"'
Files with a query that contains the "\" character (e.g., "\authors")	fullText contains '\\authors'
Files with ID within a collection, e.g. parents collection	'1234567' in parents
Files in an application data folder in a collection	'appDataFolder' in parents
Files for which user "test@example.org" has write permission	'test@example.org' in writers
Files for which members of the group "group@example.org" have write permission	'group@example.org' in writers
Files modified after a given date	modifiedTime > '2012-06-04T12:00:00' // default time zone is UTC
Files shared with the authorized user with "hello" in the name	sharedWithMe and name contains 'hello'
Files that have not been shared with anyone or domains (only private, or shared with specific users or groups)	visibility = 'limited'
Image or video files modified after a specific date	modifiedTime > '2012-06-04T12:00:00' and (mimeType contains 'image/' or mimeType contains 'video/')


### **OpenAPI スキーマの例**

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。これは、あなたのGPTの機能として含めることができる例を提供します。

質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/getting-started)をチェックして、このステップがどのように機能するかを確認してください。また、OpenAIがActionsのサポートのために作成したCustomGPTである[ActionsGPT](https://chatgpt.com/g/g-TYEliDU6A-actionsgpt)も試してみてください。3つの例は以下の通りです：

* **List Files**: これはあなたのドライブ内のファイルをリストアップするコアアクションです。この中には、`q`、`includeItemsFromAllDrives`、`supportsAllDrives`などのいくつかのパラメータが含まれています
* **Get Metadata**: リストが機能しない場合、特定の結果に基づくバックアップとして提供できます - 例えば、ユーザーが「先週のミーティング」などで検索を試みる場合など
* **Export**: バイトコンテンツでエクスポートします。詳細については、[https://developers.google.com/drive/api/reference/rest/v3/files/export](https://developers.google.com/drive/api/reference/rest/v3/files/export)を参照してください

一般的に、'get'が使用される場合、モデルはファイルのダウンロードを試みますが、これは望ましくない場合があります。そのため、代わりにExportが推奨されます。

In [ ]:

{
  "openapi": "3.1.0",
  "info": {
    "title": "Google Drive API",
    "description": "API for interacting with Google Drive",
    "version": "1.0.0"
  },
  "servers": [
    {
      "url": "https://www.googleapis.com/drive/v3"
    }
  ],
  "paths": {
    "/files": {
      "get": {
        "operationId": "ListFiles",
        "summary": "List files",
        "description": "Retrieve a list of files in the user's Google Drive.",
        "parameters": [
          {
            "name": "q",
            "in": "query",
            "description": "Query string for searching files.",
            "required": false,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "includeItemsFromAllDrives",
            "in": "query",
            "description": "Whether both My Drive and shared drive items should be included in results.",
            "required": false,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "supportsAllDrives",
            "in": "query",
            "description": "Whether the requesting application supports both My Drives and shared drives.",
            "required": false,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "pageSize",
            "in": "query",
            "description": "Maximum number of files to return.",
            "required": false,
            "schema": {
              "type": "integer",
              "default": 10
            }
          },
          {
            "name": "pageToken",
            "in": "query",
            "description": "Token for continuing a previous list request.",
            "required": false,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "fields",
            "in": "query",
            "description": "Comma-separated list of fields to include in the response.",
            "required": false,
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A list of files.",
            "content": {
              "application/json": {
                "schema": {
                  "type": "object",
                  "properties": {
                    "kind": {
                      "type": "string",
                      "example": "drive#fileList"
                    },
                    "nextPageToken": {
                      "type": "string",
                      "description": "Token to retrieve the next page of results."
                    },
                    "files": {
                      "type": "array",
                      "items": {
                        "type": "object",
                        "properties": {
                          "id": {
                            "type": "string"
                          },
                          "name": {
                            "type": "string"
                          },
                          "mimeType": {
                            "type": "string"
                          }
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    },
    "/files/{fileId}": {
      "get": {
        "operationId": "getMetadata",
        "summary": "Get file metadata",
        "description": "Retrieve metadata for a specific file.",
        "parameters": [
          {
            "name": "fileId",
            "in": "path",
            "description": "ID of the file to retrieve.",
            "required": true,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "fields",
            "in": "query",
            "description": "Comma-separated list of fields to include in the response.",
            "required": false,
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Metadata of the file.",
            "content": {
              "application/json": {
                "schema": {
                  "type": "object",
                  "properties": {
                    "id": {
                      "type": "string"
                    },
                    "name": {
                      "type": "string"
                    },
                    "mimeType": {
                      "type": "string"
                    },
                    "description": {
                      "type": "string"
                    },
                    "createdTime": {
                      "type": "string",
                      "format": "date-time"
                    }
                  }
                }
              }
            }
          }
        }
      }
    },
    "/files/{fileId}/export": {
      "get": {
        "operationId": "export",
        "summary": "Export a file",
        "description": "Export a Google Doc to the requested MIME type.",
        "parameters": [
          {
            "name": "fileId",
            "in": "path",
            "description": "ID of the file to export.",
            "required": true,
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "mimeType",
            "in": "query",
            "description": "The MIME type of the format to export to.",
            "required": true,
            "schema": {
              "type": "string",
              "enum": [
                "application/pdf",
                "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
                "text/plain"
              ]
            }
          }
        ],
        "responses": {
          "200": {
            "description": "The exported file.",
            "content": {
              "application/pdf": {
                "schema": {
                  "type": "string",
                  "format": "binary"
                }
              },
              "application/vnd.openxmlformats-officedocument.wordprocessingml.document": {
                "schema": {
                  "type": "string",
                  "format": "binary"
                }
              },
              "text/plain": {
                "schema": {
                  "type": "string",
                  "format": "binary"
                }
              }
            }
          },
          "400": {
            "description": "Invalid MIME type or file ID."
          },
          "404": {
            "description": "File not found."
          }
        }
      }
    }
  }
}

## **認証手順**

以下は、このサードパーティアプリケーションとの認証を設定する手順です。質問がありますか？[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をチェックして、この手順がどのように動作するかの詳細を確認してください。


### **事前手順**

ChatGPTで認証を設定する前に、アプリケーションで以下の手順を実行してください。



* Google Cloud Consoleにアクセス
* 有効なAPI & サービスに移動し、Google Drive APIを有効化


![alt_text](../../../images/gptactions_gd_api_services_pin.png "api_and_services")

![alt_text](../../../images/gptactions_gd_nav_to_enabled_api.png "api_lib")


* 検索バーで、Google Drive APIを検索：

![alt_text](../../../images/gptactions_gd_search_google_drive_api.png "gpt_actions")


* 新しいOAuth認証情報を作成（または既存のものを使用）。OAuth認証情報画面を設定していない場合は、それを行う必要があります。


![alt_text](../../../images/gptactions_gd_oauth_consent_screen.png "oauth_consent")




* このプロセスでは、適切な権限へのアクセスを許可し、テストが有効な場合はプライマリテスターをテストメールとして設定し、OAuthレート制限を設定する必要があります。
* 次に、認証情報に移動し、「+ 認証情報を作成」をクリックして「認証情報を作成」をクリックします。以下は、すでに設定されている場合のこの画面の例です。


![alt_text](../../../images/gptactions_gd_go_to_create_credentials.png "creds")




* OAuth Client IDとClient Secretを見つけて、両方の値を安全に保存してください（以下のスクリーンショットを参照）



![alt_text](../../../images/gptactions_gd_oauthcid_and_csecret.png "id and secret")



### **ChatGPTでの設定**

ChatGPTで「認証」をクリックし、**「OAuth」**を選択します。以下の情報を入力してください。

* **Client ID**: 上記の手順で取得したClient IDを使用
* **Client Secret**: 上記の手順で取得したClient Secretを使用
* **Authorization URL**: [https://accounts.google.com/o/oauth2/auth](https://accounts.google.com/o/oauth2/auth)
* **Token URL**: [https://oauth2.googleapis.com/token](https://oauth2.googleapis.com/token)
* **Scope**: [https://www.googleapis.com/auth/drive](https://www.googleapis.com/auth/drive.readonly)
    * **注意**: より詳細な有効スコープのリストについては、[GoogleのOAuth 2.0ガイド](https://developers.google.com/identity/protocols/oauth2/scopes)を参照してください。
* **Token**: Default (POST)
* **Privacy Policy**: [https://policies.google.com/privacy?hl=en-US](https://policies.google.com/privacy?hl=en-US)


### **事後手順**

ChatGPTで認証を設定した後、アプリケーションで以下の手順に従ってActionを完了してください。



* GPT ActionからコールバックURLをコピー


![alt_text](../../../images/gptactions_gd_callbackurl_from_gpt_action.png "callback")




* 「承認済みリダイレクトURI」に、コールバックURLを追加


![alt_text](../../../images/gptactions_gd_authorized_redirect_uris.png "image_tooltip")



### **FAQ & トラブルシューティング**



* _コールバックURLエラー:_ ChatGPTでコールバックURLエラーが発生した場合は、上記のスクリーンショットに注意深く従ってください。アクションが正しく認証されるように、コールバックURLを直接GCPに追加する必要があります。

_優先して欲しい統合機能はありますか？統合にエラーはありますか？GitHubでPRやissueを提出していただければ、確認いたします。_




<!-- watermark --><div style="background-color:#FFFFFF"><p style="color:#FFFFFF; font-size: 1px">gd2md-html: xyzzy Mon Aug 12 2024</p></div>